In [18]:
!pip install postgres

## Moodle Class
The class handles connection to the database and making retrivals. 

In [137]:
import pandas as pd
from sqlalchemy import create_engine

# The class handles connection to the database and making retrivals.
class Moodle():
    def __init__(self):
        self.cnx = None
    def connect_db(self,address,port,username,password, name):
        POSTGRES_ADDRESS = address ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
        POSTGRES_PORT = '5432'
        POSTGRES_USERNAME = username ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
        POSTGRES_PASSWORD = password ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
        POSTGRES_DBNAME = name ## CHANGE THIS TO YOUR DATABASE NAME
        
        # A long string that contains the necessary Postgres login information
        postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
        # Create the connection
        self.cnx = create_engine(postgres_str)
        
    def get_num_records_of_table(self, table):
        return pd.read_sql_query('''SELECT COUNT(*) FROM '''+table+''' ;''', self.cnx)
        
    def get_num_tables(self):
        return pd.read_sql_query('''select count(*) from information_schema.tables where table_schema = 'public';''', self.cnx)
    
    def get_num_quiz_submissions_by_hour(self):
        return pd.read_sql_query('''select count(id), EXTRACT(HOUR FROM to_timestamp(timecreated)) AS HOUR from mdl_logstore_standard_log where action='submitted' AND component='mod_quiz' group by EXTRACT(HOUR FROM to_timestamp(timecreated));''', self.cnx)
    def get_num_events_by_user(self):
        return pd.read_sql_query('''select userid, sum(case when action = 'viewed' then 1 else 0 end) as viewed,sum(case when action = 'uploaded' then 1 else 0 end) as uploaded ,sum(case when action = 'updated' then 1 else 0 end) as updated,sum(case when action = 'answered' then 1 else 0 end) as answered ,sum(case when action = 'abandoned' then 1 else 0 end) as abandoned, sum(case when action = 'submitted' then 1 else 0 end) as Submitted, sum(case when action = 'searched' then 1 else 0 end) as searched, sum(case when action = 'attempted' then 1 else 0 end) as attempted from mdl_logstore_standard_log group by userid;''', self.cnx)

    def get_overall_grades(self):
        return pd.read_sql_query('''SELECT userid, sum(finalgrade) FROM mdl_grade_grades GROUP BY userid ORDER BY userid;''', self.cnx)
    def get_num_forum_posts(self):
        return pd.read_sql_query('''SELECT COUNT(*) FROM mdl_forum_posts ;''', self.cnx)
    

## Instansiate Moodle object

In [138]:
moodle = Moodle()

## Make Connection

In [139]:
POSTGRES_ADDRESS = '127.0.0.1'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'nahom' 
POSTGRES_DBNAME = 'Moodle' 
moodle.connect_db(POSTGRES_ADDRESS, POSTGRES_PORT, POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_DBNAME )

### Get number of tables in database

In [122]:
moodle.get_num_tables().values[0][0]

448

### Get number of records in important tables

### mdl_logstore_standard_log

In [124]:
moodle.get_num_records_of_table('mdl_logstore_standard_log').values[0][0]

417554

### mdl_grade_grades

In [148]:
moodle.get_num_records_of_table('mdl_grade_grades').values[0][0]

3643

### mdl_context

In [146]:
moodle.get_num_records_of_table('mdl_context').values[0][0]

4359

### mdl_user

In [145]:
moodle.get_num_records_of_table('mdl_user').values[0][0]

1052

### mdl_course

In [144]:
moodle.get_num_records_of_table('mdl_course').values[0][0]

15

### mdl_course_modules

In [143]:
moodle.get_num_records_of_table('mdl_course_modules').values[0][0]

290

### Get number of submissions by hour  
This counts the number of submissions made each hour

In [125]:
moodle.get_num_quiz_submissions_by_hour()

,count,hour
0,46,10.0
1,26,0.0
2,57,20.0
3,46,22.0
4,68,17.0
5,53,23.0
6,88,13.0
7,95,12.0
8,90,15.0
9,14,5.0


### Count of log events
Count of log events per user for the following verbs: ['loggedin', 'viewed',
'started', ,'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered',
'attempted', 'abandoned']

In [141]:
moodle.get_num_events_by_user().sample(10)

,userid,viewed,uploaded,updated,answered,abandoned,submitted,searched,attempted
580,670,12,0,0,0,0,0,0,0
1040,601,12,0,0,0,0,0,0,0
468,232,80,0,2,0,0,1,0,0
898,1044,9,0,0,0,0,0,0,0
997,874,11,0,0,0,0,0,0,0
494,928,11,0,0,0,0,0,0,0
721,621,12,0,0,0,0,0,0,0
396,863,11,0,0,0,0,0,0,0
417,1001,23,0,5,0,0,4,0,0
124,325,41,0,0,0,0,0,0,0


### Overall grade of learners

In [114]:
moodle.get_overall_grades()

,userid,sum
0,2,NaN
1,3,200.00000
2,5,400.66666
3,7,200.00000
4,9,636.00000
...,...,...
402,991,200.00000
403,1019,200.00000
404,1025,2700.00000
405,1027,3124.00000


### Number of forum posts

In [131]:
moodle.get_num_forum_posts().values[0][0]

131